# Chat con encriptación de clave pública RSA     
## Segunda entrega de proyecto
## Matemáticas Discretas II

Juan Camilo García Martínez 

Cristian Alejandro Chávez Becerra

## 1. Introducción

Las
plataformas de chat más populares suelen usar servidores intermediarios que se encargan de la
transmisión de los datos. Si bien prometen no espiar o siquiera ser capaces de hacerlo debido al
uso de encriptación extremo a extremo, la naturaleza privativa de su código fuente no nos
asegura nada.
Como alternativa a estos medios centralizados, se puede hacer el traspaso de información de
forma más directa, como por ejemplo, usando sockets, que no requieren un servidor intermedio.
El problema es entonces la seguridad; el mensaje pasa por las diferentes capas de red tal y
como se envió inicialmente y está sujeto a intrusión por parte de terceros. Es necesaria una
forma de encriptar el mensaje.

## 2. Materiales y Métodos


 





###RSA
La encriptación RSA está basada en 4 pilares esenciales de la teoría de números:
 
1. La cardinalidad del conjunto de números menores que $n$ que son asimismo coprimos con n esta dada por la función de Euler' $φ$. Si este numero n es primo se tiene que $\varphi(n) = n-1$
2. Si $n=pq$ siendo $p$ y $q$ dos números primos, entonces $\varphi(n) = (p-1)(q-1)$
3. Por el pequeño teorema de Fermat, se sabe que si $a$ es un número entero mayor que $0$ y $p$ un número primo, se tiene que cumplir $a^{p-1} = 1\pmod{p}$
4. Por el teorema de Euler, si $mcd(n, a) = 1$, entonces $a^{\varphi(n)} = 1 \pmod{n}$
 
 
El algoritmo de encriptación consiste en:
 
 
Suponga que un usuario $A$ , quiere recibir mensajes encriptados. Para ello genera dos números primos $p$ y $q$ y luego asigna su producto a una variable $n = pq$
 
 Se escoge un valor $e$ usualmente llamado el exponente de manera que el $mcd(\varphi(n), e) = 1$. 
 
 
El par $(n, e)$ se conoce como la llave pública, como su nombre lo indica los usuarios pueden hacer uso de la llave pública para recibir mensajes, pero no deben revelar el valor de los primos $p$ y $q$, pues le permitiría a un intruso saber $\varphi(n)$ y así también la llave privada, $d$, donde $de = 1\mod{\varphi(n)}$.
 
Los usuarios que quieran enviar mensajes encriptados a un usuario $A$, pueden hacerlo mediante el uso  de la llave pública de este como sigue: 
 
El usuario $A$ envía su llave publica $(n_{A}, e_{A})$ a otro usuario, $B$. Este envío puede pasar por el mismo canal inseguro por el que los datos encriptados pasaran luego.
 
El usuario $B$ usa la llave pública de $A$ para encriptar un texto $x$:
 
$y = x^{e_{A}} \pmod{n_{A}}$
 
Luego le envía $y$ a $A$.
 
El usuario $A$ recibe esta $y$ y por medio del inverso multiplicativo y también llave privada $d$, calculada desde el principio, realiza la operación.
 
$y^{d_{A}} \pmod{n_{A}} = x$
 
Cualquier otro usuario $C$ con acceso al canal de información que no sepa $p,q,\varphi(n)$ o $d$ tiene que calcular por algún método (que tiene un altisimo costo computacional) el valor de estas variables para poder descifrar el $x$ original enviado por $B$
 
En nuestro programa, una $x$  es el valor ASCII de un carácter perteneciente a una cadena. Para todo carácter de la cadena se hace el proceso anterior y uno por uno se añade junto a un delimitador a una cadena de texto, que es el mensaje de chat.

### Exponenciación binaria.
 
Es claro que la operación más repetida tras generar las llaves públicas es la exponenciación: Se hace uso de ella tanto para encriptar $x$ como para desencriptar $y$.

Si siguieramos un algoritmo lineal tipo:

Calcular $x^{n}$

$x \times x = x^{2}$

$x \times x^2 = x^{3}$

$\vdots$

$x \times x^{n-1} = x^{n}$

Tendremos que efectuar la misma operación $n-1$ veces. En RSA, donde los números primos $p$ y $q$ y por lo tanto los exponentes $d$ o $e$ pueden ser cercanos a longitudes $2^{256}$, $2^{512}$, $2^{1024}$, $2^{2048}$ la cantidad de operaciones es enorme; solo para el primer caso tendriamos que hacer $115792089237316195423570985008687907853269984665640564039457584007913129639936-1$ operaciones.


Podemos optimizar el cálculo de estos números por medio de un algoritmo conocido como *exponenciación binaria* o *square-and-multiply* que hace uso de la representación binaria del exponente. Presentamos una implementación en Python basada en el algoritmo encontrado en Understaunding Cryptography, pag 182.

```
base: x
exponente: H expresado en binario = h02^0+...+ht2^t
parametros: base, exponente, modulo

Algoritmo:
Haga r = x
Para i=t−1 hasta 0
  r=r*r modn 
  Si hi=1
    r = r· x modn2
retorne x
```



Como se observa se recorren los $20$ digitos binarios de la representación binaria del exponente decimal $654982$: $10011111111010000110_{2}$. Eso es $\lceil \log_{2}(exponente) \rceil$

Hemos reducido la complejidad  de $\mathcal{O}(n)$ a $\mathcal{O}(\log{}n)$.




In [ ]:
#Ejemplo 7^(654982)

def fast_exponentiation(base, exponent):
        '''
        Exponentiation-by-squaring algorithm.
        '''
        binary_exponent = f"{exponent:b}" #Obtenga la representación binaria de 654982: 10011111111010000110_2
        result = base
        for binary_digit in binary_exponent[1:]: #Recorra todos los digitos binarios en la representación menos el de menor peso.
            result = result * result  #Para todo caso haga result = result^2
            if(binary_digit==1):
                result = result * base #Si el digito actual es 1, tambien multiplique por la base original.
        return result

print(fast_exponentiation(7, 654982))

...


In [ ]:
#Debido a que estos numeros con potencias grandes van a ser evaluados siguiendo una aritmetica modular,
#se usa esta funcion en el programa

def fast_exponentiation(base, exponent, modulus):
        '''
        Exponentiation-by-squaring algorithm.
        '''
        binary_exponent = f"{exponent:b}"
        result = base
        for binary_digit in binary_exponent[1:]:
            result = (result * result) % modulus 
            if(int(binary_digit)==1):
                result = (result * base) % modulus  
        return result

print(fast_exponentiation(7, 654982, 15))

4


###Reducción por Teorema Chino del resto

Se puede optimizar tambien la velocidad de exponenciacíon de desencriptacion, eso es  $x^{d}$ usando el *teorema chino del resto* 

**Teorema Chino del Resto** Suponga $p$ y $q$ son dos coprimos 2 a 2. Luego

$x \equiv a \mod{pq} $  si y solo si $x \equiv a \mod{p} $ y $x \equiv a \mod{q} $

Como $n =pq$ y $p, q$ son ambos primos podemos hacer uso del teorema para reducir los exponentes y el numero de operaciones de exponenciación.

**Algoritmo**

Reducimos el elemento $x$ en el modulo de factores $p$ y $q$

$x_{p}\equiv x\mod{p}$

$x_{q}\equiv x\mod{q}$

Con las versiones reducidas se hace:

$y_{p}  = (x_{p})^{d_{p}} \mod{p}$

$y_{q}  = (x_{q})^{d_{q}} \mod{q}$

Donde estos nuevos exponentes son

$d_{p}  = d \mod{p-1}$

$d_{q}  = d \mod{q-1}$

El paso final proviene del teorema tambien.

$y \equiv [qc_{p}]y_{p} + [qc_{q}]y_{q} \mod{n}$

Donde estos coefiencites $c_{p}$ y $c_{q}$ son calculados como 

$c_{p}  = q^{-1} \mod{p}$

$c_{q}  = p^{-1} \mod{q}$

Estos coeficientes pueden ser computados desde la generacion de $n$ y almacenados hasta el cambio de primos.

In [ ]:
#En el programa hacemos uso de este algoritmo para exponenciar x.

def crt_domain_reduction(self, x):
        x_p = x % self.p
        x_q = x % self.q

        d_p = self.d % (self.p-1)
        d_q = self.d % (self.q-1)

        y_p = RSA.fast_exponentiation(x_p, d_p, self.p)
        y_q = RSA.fast_exponentiation(x_q, d_q, self.q)

        c_p = self.cp
        c_q = self.cq

        return ((self.q*c_p)*y_p + (self.p*c_q)*y_q)% self.n

## 3. Resultados

Para esta segunda entrega se presenta una implementacion de RSA y algunos metodos que optimizan la velocidad de las operaciones que requiere. 


Se ofrece una demostracion de la encriptacíon hecha por un programa rsa.py que simula la encriptación de datos entre dos usuarios $Alice$, $Bob$ y de un intruso $Chuck$ que no conoce las llaves privadas de ninguno. 

El sistema desarrollado hasta ahora es claro solo la versión "Textbook" de RSA y  puede ser mejorado usando tecnicas como "padding". En la practica de mensajeria, RSA no se usa para codificar el texto, si no para codificar y una sesión que use cifrado simetrico.

El desarrollo del programa puede ser consultado en nuestro repositorio.

https://github.com/AlejandroUN/rsaEncryptionChat






In [ ]:
import random

class RSA:


    def __init__(self, p, q):

        #soon get random p and q using Miller-Rabi
        self.p = p
        self.q = q
        self.n = self.p*self.q
        self.totient = RSA.find_totient(self.p, self.q)
        self.e = self.find_e(self.totient)
        self.d = RSA.find_inverse(self.e, self.totient)
        self.public_key = (self.n, self.e)
        self.private_key = self.d

        self.cp = RSA.find_inverse(self.q, self.p)
        self.cq = RSA.find_inverse(self.p, self.q)
    
    def find_totient(p, q):
        return ((p-1)*(q-1))

    def find_e(self, totient):
        
        for integer in range(3, self.totient):
            if(RSA.gcd(self.totient, integer) == 1):
                return integer

    @staticmethod
    def isPrime(n): 
        i = 2
        while(i * i <= n): 
            if (n % i == 0): 
                return False; 
            i = i + 1; 
        return True

    @staticmethod
    def gcd(a, b):
        if a%b == 0:
            return b
        else:
            return RSA.gcd(b, a%b)

    def find_inverse(e, totient):
        for x in range(2, totient):
            if (e*x)%totient == 1:
                return x
        return 0

    def fast_exponentiation(base, exponent, modulus):
        '''
        Exponentiation-by-squaring algorithm.
        '''
        binary_exponent = f"{exponent:b}"
        result = base
        for binary_digit in binary_exponent[1:]:
            result = (result * result) % modulus 
            if(int(binary_digit)==1):
                result = (result * base) % modulus  
        return result

    #using fast_exponentiation
    def codificate_message(self, n, e, message):
        coded_message = ""
        for x in message:
            x_ascii = ord(x)%n
            encoded_ascii = RSA.fast_exponentiation(x_ascii, e, n)
            coded_message += str(encoded_ascii) + "/"
        return coded_message[:-1]

     #using crt
    def decode_message(self, coded_message, crt = True, modular=False):
        decoded_message = "" 
        coded_message = coded_message.split("/")
        for y in coded_message:
            if crt: result_modular_exp = self.crt_domain_reduction(int(y))
            if modular: result_modular_exp = RSA.fast_exponentiation(int(y), self.d, self.n)
            decoded_message += str(chr(result_modular_exp))
        return decoded_message

    def crt_domain_reduction(self, x):
        x_p = x % self.p
        x_q = x % self.q

        d_p = self.d % (self.p-1)
        d_q = self.d % (self.q-1)

        y_p = RSA.fast_exponentiation(x_p, d_p, self.p)
        y_q = RSA.fast_exponentiation(x_q, d_q, self.q)

        c_p = self.cp
        c_q = self.cq

        return ((self.q*c_p)*y_p + (self.p*c_q)*y_q)% self.n

if __name__ == "__main__":          

    Alice = RSA(191, 193)
    Bob = RSA(241,239)
    Chuck = RSA(211, 223)

    print("Bob n {}, e {}, totient {}, d {} cp {} cq {}".format(Bob.n, Bob.e, Bob.totient, Bob.d, Bob.cp, Bob.cq))
    print("Alice n {}, e {}, totient {}, d {} cp {} cq {}".format(Alice.n, Alice.e, Alice.totient, Alice.d, Alice.cp, Alice.cq))
    print("Chuck n {}, e {} , totient {}, d {} cp {} cq {}".format(Chuck.n, Chuck.e, Chuck.totient, Chuck.d, Chuck.cp, Chuck.cq))

    #Alice sends a message to Bob
    #somehow send Bob's pk to Alice at the start, use Bob's pk to encrypt

    alice_message = Alice.codificate_message(Bob.n, Bob.e, "You now quasimodo predicted all this")
    print("Encrypted msg by alice ", alice_message)
    # Bob uses d to decrypt
    bob_decrypt = Bob.decode_message(alice_message)
    print("Bob decrypts Alice msg: ", bob_decrypt)

    
    bob_message = Bob.codificate_message(Alice.n, Alice.e, "who did what? nostradamus and notre dame are two different things completely.")
    print("encrypted msg by bob ", bob_message)
    alice_decrypt = Alice.decode_message(bob_message)
    print("Alice decrypts Bob's msg", alice_decrypt)

    #Now Chuck tries to read their messages using his private key
    chuck_decrypts_alice = Chuck.decode_message(alice_message)
    chuck_decrypts_bob = Chuck.decode_message(bob_message)

    print("Chuck tries to decrypt alice's msg", chuck_decrypts_alice)
    print("Chuck tries to decrypt bob's msg", chuck_decrypts_bob)
    print("Chuck thinks buth alice and bob are from china and soon gives up")

Bob n 57599, e 11, totient 57120, d 20771 cp 120 cq 120
Alice n 36863, e 7, totient 36480, d 10423 cp 96 cq 96
Chuck n 47053, e 11 , totient 46620, d 21191 cp 88 cq 130
Encrypted msg by alice  7293/11371/8571/30976/21876/11371/47742/30976/9849/8571/26260/15982/57475/37482/11371/52351/11371/30976/46216/13364/32397/52351/57475/15835/27280/32397/52351/30976/26260/55139/55139/30976/27280/23084/57475/15982
Bob decrypts Alice msg:  You now quasimodo predicted all this
encrypted msg by bob  15897/11560/23414/30972/3263/14635/3263/30972/15897/11560/3376/25172/35449/30972/30998/23414/9635/25172/34302/3376/3263/3376/30217/5701/9635/30972/3376/30998/3263/30972/30998/23414/25172/34302/17402/30972/3263/3376/30217/17402/30972/3376/34302/17402/30972/25172/15897/23414/30972/3263/14635/20227/20227/17402/34302/17402/30998/25172/30972/25172/11560/14635/30998/24458/9635/30972/33256/23414/30217/660/31760/17402/25172/17402/31760/2679/10663
Alice decrypts Bob's msg who did what? nostradamus and notre dame ar


## 4. Conclusiones

Se evidencia el uso de conceptos basicos de la teoría de números para desarollar un sencillo sistema de encriptación. 

Conceptos que pueden sonar tan abstractos e incluso irrelevantes a primera vista son la base del desarrolllo moderno.

Con conceptos básicos de teoría de números y de software, principalmente de la aritmética modular, se puede hacer un chat de encriptación lo cual de paso a la razón a imaginar las aplicaciones que tendrán conceptos más avanzados de este campo de las matemáticas utilizando mejores herramientas computacionales.




## 5. Referencias

- Paar, C. and Pelzl, J., 2011. Understanding Cryptography. 1st ed. Berlin: Springer.

- Hoffstein, J., Pipher, J., & Silverman, J. (2010). An introduction to mathematical cryptography. New York, NY: Springer.

- Number Theory and Cryptography. Recuperado de http://pi.math.cornell.edu/~mec/2008-2009/Anema/numbertheory/rsa.html
